# Model Output Monitoring Demo
## Prediction Drift and Performance Detection

This notebook demonstrates model output monitoring including:
- Prediction distribution drift detection
- Recommendation rate stability
- Model output anomalies
- Performance monitoring (when outcomes available)

**Use Case:** Monitor Random Forest uplift model predictions for the Intervention Recommendation Engine

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from prediction_monitor import PredictionMonitor
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Libraries imported successfully")

## 1. Load Baseline and Daily Predictions

Predictions from Random Forest T-Learner model (two models for uplift estimation)

In [ ]:
# Load baseline predictions (first 14 days)
baseline_predictions = pd.read_parquet('./data/baseline_predictions.parquet')

print(f"Baseline predictions: {len(baseline_predictions):,} rows")
print(f"\nColumns:")
print(list(baseline_predictions.columns))
print(f"\nSummary statistics:")
baseline_predictions[['model_treated_prob', 'model_control_prob', 'uplift_score', 'recommendation']].describe()

In [ ]:
# Load all 30 days of predictions
import glob
prediction_files = sorted(glob.glob('./data/predictions_*.parquet'))

daily_predictions = []
for file in prediction_files:
    df = pd.read_parquet(file)
    daily_predictions.append(df)

print(f"✓ Loaded {len(daily_predictions)} days of predictions")
print(f"  - Each day: ~{len(daily_predictions[0]):,} predictions")
print(f"\nSample from Day 1:")
daily_predictions[0].head()

## 2. Initialize Prediction Monitor

In [ ]:
# Initialize monitor with baseline
pred_monitor = PredictionMonitor(baseline_predictions)

print(f"✓ Prediction monitor initialized")
print(f"\nBaseline statistics:")
print(json.dumps(pred_monitor.baseline_stats, indent=2))

## 3. Run Daily Prediction Monitoring

In [ ]:
# Run monitoring for all 30 days
pred_monitoring_results = []

for day_idx, df in enumerate(daily_predictions):
    result = pred_monitor.monitor_daily_predictions(df)
    result['day'] = day_idx + 1
    pred_monitoring_results.append(result)

print(f"✓ Prediction monitoring completed for {len(pred_monitoring_results)} days")
print(f"\nDaily Summary:")
print("Day | Status | Drift | Alerts | Rec Rate")
print("-" * 50)
for result in pred_monitoring_results:
    summary = result['summary']
    stats = result['current_stats']
    drift = result['drift_test']['drifted']
    status = '🔴' if summary['high_severity_alerts'] > 0 else '🟡' if summary['total_alerts'] > 0 else '🟢'
    drift_icon = '⚠️' if drift else '✓'
    print(f" {result['day']:2d} |   {status}    |  {drift_icon}   |   {summary['total_alerts']:2d}   | {stats['recommendation_rate']*100:5.1f}%")

## 4. Visualize Prediction Monitoring Metrics

In [ ]:
# Extract time series
days = [r['day'] for r in pred_monitoring_results]
mean_uplift = [r['current_stats']['mean_uplift'] for r in pred_monitoring_results]
rec_rate = [r['current_stats']['recommendation_rate'] * 100 for r in pred_monitoring_results]
ks_pvalues = [r['drift_test']['ks_p_value'] for r in pred_monitoring_results]
alerts = [r['summary']['total_alerts'] for r in pred_monitoring_results]

# Create dashboard
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Prediction Monitoring Dashboard - 30 Days', fontsize=16, fontweight='bold')

# Plot 1: Mean uplift score over time
baseline_mean = pred_monitor.baseline_stats['mean_uplift']
axes[0, 0].plot(days, mean_uplift, marker='o', color='blue', linewidth=2, label='Current')
axes[0, 0].axhline(y=baseline_mean, color='green', linestyle='--', linewidth=2, label='Baseline')
axes[0, 0].set_xlabel('Day')
axes[0, 0].set_ylabel('Mean Uplift Score')
axes[0, 0].set_title('Mean Uplift Score Over Time')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Recommendation rate over time
baseline_rec_rate = pred_monitor.baseline_stats['recommendation_rate'] * 100
axes[0, 1].plot(days, rec_rate, marker='s', color='orange', linewidth=2, label='Current')
axes[0, 1].axhline(y=baseline_rec_rate, color='green', linestyle='--', linewidth=2, label='Baseline')
axes[0, 1].axhline(y=baseline_rec_rate + 10, color='red', linestyle=':', alpha=0.5, label='Alert threshold')
axes[0, 1].axhline(y=baseline_rec_rate - 10, color='red', linestyle=':', alpha=0.5)
axes[0, 1].set_xlabel('Day')
axes[0, 1].set_ylabel('Recommendation Rate (%)')
axes[0, 1].set_title('Patients Recommended for Intervention')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Distribution drift (KS test p-value)
axes[1, 0].plot(days, ks_pvalues, marker='o', color='purple', linewidth=2)
axes[1, 0].axhline(y=0.01, color='red', linestyle='--', label='Significance threshold (p=0.01)')
axes[1, 0].set_xlabel('Day')
axes[1, 0].set_ylabel('KS Test p-value')
axes[1, 0].set_title('Uplift Score Distribution Drift')
axes[1, 0].set_yscale('log')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Alerts over time
axes[1, 1].bar(days, alerts, color='red', alpha=0.7)
axes[1, 1].set_xlabel('Day')
axes[1, 1].set_ylabel('Alert Count')
axes[1, 1].set_title('Daily Alerts')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('./prediction_monitoring_dashboard.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Visualizations saved")

## 5. Uplift Score Distribution Comparison

In [ ]:
# Compare uplift distributions: baseline vs current
day_1 = daily_predictions[0]
day_15 = daily_predictions[14]
day_30 = daily_predictions[29]

fig, axes = plt.subplots(1, 3, figsize=(16, 5))
fig.suptitle('Uplift Score Distribution Over Time', fontsize=14, fontweight='bold')

# Day 1
axes[0].hist(day_1['uplift_score'], bins=50, color='blue', alpha=0.7, edgecolor='black')
axes[0].axvline(day_1['uplift_score'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {day_1["uplift_score"].mean():.3f}')
axes[0].set_xlabel('Uplift Score')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Day 1 (Baseline)')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# Day 15
axes[1].hist(day_15['uplift_score'], bins=50, color='orange', alpha=0.7, edgecolor='black')
axes[1].axvline(day_15['uplift_score'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {day_15["uplift_score"].mean():.3f}')
axes[1].set_xlabel('Uplift Score')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Day 15')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

# Day 30
axes[2].hist(day_30['uplift_score'], bins=50, color='green', alpha=0.7, edgecolor='black')
axes[2].axvline(day_30['uplift_score'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {day_30["uplift_score"].mean():.3f}')
axes[2].set_xlabel('Uplift Score')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Day 30')
axes[2].legend()
axes[2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('./uplift_distribution_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Alert Analysis

In [ ]:
# Extract all prediction alerts
pred_alerts = []
for result in pred_monitoring_results:
    for alert in result['alerts']:
        pred_alerts.append({
            'day': result['day'],
            'severity': alert['severity'],
            'metric': alert['metric'],
            'message': alert['message']
        })

if len(pred_alerts) > 0:
    pred_alerts_df = pd.DataFrame(pred_alerts)
    
    print(f"Total prediction alerts: {len(pred_alerts_df)}")
    print(f"\nAlerts by severity:")
    print(pred_alerts_df['severity'].value_counts())
    print(f"\nAlerts by metric:")
    print(pred_alerts_df['metric'].value_counts())
    
    if len(pred_alerts_df[pred_alerts_df['severity'] == 'HIGH']) > 0:
        print(f"\n" + "="*80)
        print("HIGH SEVERITY ALERTS:")
        print("="*80)
        high_alerts = pred_alerts_df[pred_alerts_df['severity'] == 'HIGH']
        for idx, row in high_alerts.iterrows():
            print(f"Day {row['day']:2d} | {row['metric']:25s} | {row['message']}")
else:
    print("✓ No alerts detected - predictions are stable!")

## 7. Summary & Key Findings

### Detection Capabilities:

✅ **Successfully Monitors:**
- Uplift score distribution stability (KS test)
- Mean uplift score shifts (z-score)
- Recommendation rate changes
- Extreme predictions (scores <0 or >1)
- Model component drift (treated/control model outputs)

### Production Recommendations:

1. **Daily Monitoring:** Run after daily scoring completes
2. **Alerting:** 
   - HIGH: Mean shift >3 std devs OR extreme predictions >1%
   - MEDIUM: Distribution drift OR rec rate change >10pp
3. **Model Refresh:** Consider retraining if alerts persist >5 days
4. **Root Cause:** Check input data drift first (likely upstream cause)
5. **Model Health:** Random Forest models are typically stable - alerts indicate data issues

In [ ]:
# Final summary
print("="*80)
print("PREDICTION MONITORING SUMMARY")
print("="*80)
print(f"\nMonitoring Period: 30 days")
print(f"Baseline: Days 1-14")
print(f"Model: Random Forest T-Learner (Uplift)")
print(f"\nPrediction Statistics:")
print(f"  - Total predictions: {sum(len(d) for d in daily_predictions):,}")
print(f"  - Avg daily predictions: {np.mean([len(d) for d in daily_predictions]):.0f}")
print(f"  - Avg recommendation rate: {np.mean(rec_rate):.1f}%")
print(f"\nMonitoring Results:")
if len(pred_alerts) > 0:
    pred_alerts_df = pd.DataFrame(pred_alerts)
    print(f"  - Total alerts: {len(pred_alerts_df)}")
    print(f"  - HIGH severity: {len(pred_alerts_df[pred_alerts_df['severity'] == 'HIGH'])}")
    print(f"  - MEDIUM severity: {len(pred_alerts_df[pred_alerts_df['severity'] == 'MEDIUM'])}")
else:
    print(f"  - Total alerts: 0")
print(f"  - Days with drift: {sum(1 for r in pred_monitoring_results if r['drift_test']['drifted'])}")
print(f"\nModel Stability: {'🟢 STABLE' if len(pred_alerts) == 0 else '🟡 MONITOR' if all(a['severity'] != 'HIGH' for a in pred_alerts) else '🔴 ACTION NEEDED'}")
print("\n" + "="*80)
print("✓ Prediction monitoring demo complete!")
print("="*80)